In [2]:
import sys
import os

sys.path.append('******\\visualize\\tau') # Path for tau
sys.path.append('******\\visualize\\tau\\DRL_code')# Path for DRL_code

In [3]:
import torch
from torch import nn
from torch.distributions import Normal
import torch.nn.functional as F

import matplotlib.pyplot as plt

import gym 
gym.logger.set_level(40) 
import sac # 自作モジュール(アルゴリズム)
import replay_memory # 自作モジュール

import gym_pathplan # 自作 gym 環境

device=torch.device('cuda')

In [4]:
ENV_ID = 'STLPathPlan-v1'

env = gym.make(ENV_ID)

In [5]:
class SACActor(nn.Module):

    def __init__(self, state_shape, action_shape):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(state_shape[0], 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 2 * action_shape[0]),
        )

    def forward(self, states):
        return torch.tanh(self.net(states).chunk(2, dim=-1)[0])

    def sample(self, states):
        means, log_stds = self.net(states).chunk(2, dim=-1)
        return utility.reparameterize(means, log_stds.clamp(-20, 2))

In [6]:
# 重みの読込をおこなう
baseline_path = 'tau/data/seed0/SAC_STL_actor_600000.pth'
baseline_param = torch.load(baseline_path)
baseline_policy = SACActor(env.extended_state_space.shape,env.action_space.shape).to(device)
baseline_policy.load_state_dict(baseline_param)

<All keys matched successfully>

In [7]:
# 実際に制御してみる．
evaluate_val = 1.0
state = env.reset_for_test(2.0, 1.0, 0.0)
evaluate_temp = env.evaluate_stl_formula()
evaluate_val = min(evaluate_val,evaluate_temp)
done = False
return_val = 0.0
stl_return_val = 0.0
t = 0
gamma_count = 0
GAMMA = 0.99

while (not done):
    t += 1
    env.render(mode='rgb_array')
    input_state = torch.tensor(state, dtype=torch.float, device=device).unsqueeze_(0) # DNN に入力するため torch.tensor型に
    input_state_ = input_state.cpu().detach().numpy()
    action = baseline_policy(input_state).reshape(-1)
    action_ = action.cpu().detach().numpy() # gym 上では numpy で動くので変換
    next_state, stl_reward, done, true_action = env.step_for_test(action_) # 状態遷移
    
    print("z("+str(t-1)+")="+str(input_state_))
    print("a("+str(t-1)+")="+str(action_))
    print("u("+str(t-1)+")="+str(true_action))
    
    if t == env._max_episode_steps:
        done_masked = False
    else:
        done_masked = done

    evaluate_temp = env.evaluate_stl_formula()
    evaluate_val = min(evaluate_val, evaluate_temp) # \Phi = G\phi
    stl_return_val += (GAMMA**gamma_count)*stl_reward
    gamma_count += 1
    
    # エピソードが終了した場合には，環境をリセットする．
    if done:
        t = 0
        state = env.reset()
    else:
        state = next_state
    
if evaluate_val == 1.0:
    print("この方策はSTL仕様を満たしています")
else:
    print("この方策はSTL仕様を満たしていません")
    
env.close()

z(0)=[[-0.5 -1.5  0.  -0.5 -0.5]]
a(0)=[0.93353236 0.92158806]
u(0)=[0. 0.]
z(1)=[[-0.4916025  -1.4986528   0.00513523 -0.5        -0.5       ]]
a(1)=[0.93495995 0.91733795]
u(1)=[0. 0.]
z(2)=[[-0.49157447 -1.4948258   0.00960367 -0.5        -0.5       ]]
a(2)=[0.936188  0.9148766]
u(2)=[0. 0.]
z(3)=[[-0.48207086 -1.4724227   0.01076895 -0.5        -0.5       ]]
a(3)=[0.9342061 0.9129102]
u(3)=[0. 0.]
z(4)=[[-0.48023283 -1.4780006   0.03122056 -0.5        -0.5       ]]
a(4)=[0.94125897 0.90068525]
u(4)=[0. 0.]
z(5)=[[-0.47628245 -1.482855    0.05300232 -0.5        -0.5       ]]
a(5)=[0.9486414 0.8858007]
u(5)=[0. 0.]
z(6)=[[-0.45479107 -1.4744921   0.02894192 -0.5        -0.5       ]]
a(6)=[0.93877465 0.8997116 ]
u(6)=[0. 0.]
z(7)=[[-0.4393063  -1.4718397   0.03506104 -0.5        -0.5       ]]
a(7)=[0.9400414 0.8943806]
u(7)=[0.93353236 0.92158806]
z(8)=[[-0.32627308 -1.4661278   0.13331503 -0.5        -0.5       ]]
a(8)=[0.96215105 0.83242846]
u(8)=[0.93495995 0.91733795]
z(9)=[[-0.22

z(73)=[[ 1.3694993  1.2243658  2.0908039 -0.5        0.35     ]]
a(73)=[ 0.98204935 -0.13554056]
u(73)=[0.99856025 0.03856106]
z(74)=[[1.3086464 1.2952061 2.1046348 0.5       0.34     ]]
a(74)=[-0.5411819   0.11012866]
u(74)=[0.9985383  0.02224032]
z(75)=[[1.2611821 1.3752887 2.106984  0.5       0.33     ]]
a(75)=[-0.397717    0.02208959]
u(75)=[ 0.99860466 -0.08289369]
z(76)=[[1.2185719 1.4504354 2.0906253 0.49      0.32     ]]
a(76)=[-0.25711337 -0.07631297]
u(76)=[ 0.9984359  -0.00460692]
z(77)=[[1.1830838 1.5273523 2.0992355 0.48      0.31     ]]
a(77)=[-0.29461685 -0.11410235]
u(77)=[ 0.99809694 -0.0498287 ]
z(78)=[[1.1379588 1.6037823 2.0833077 0.47      0.3      ]]
a(78)=[-0.31500986 -0.10645431]
u(78)=[ 0.9972442  -0.04190349]
z(79)=[[1.088234  1.6913624 2.0775733 0.46      0.29     ]]
a(79)=[-0.39329597 -0.10894458]
u(79)=[0.9950536  0.00239516]
z(80)=[[1.0429054 1.77854   2.0813363 0.45      0.28     ]]
a(80)=[-0.52628356 -0.13858399]
u(80)=[ 0.98204935 -0.13554056]
z(81)=[[0

z(142)=[[ 1.4446775  0.785888   1.8882273 -0.17       0.4      ]]
a(142)=[ 0.97051835 -0.04174814]
u(142)=[0.73643726 0.64285505]
z(143)=[[ 1.4170994   0.85512877  1.9476693  -0.18        0.39      ]]
a(143)=[ 0.97645795 -0.14360851]
u(143)=[0.8262807 0.6062168]
z(144)=[[ 1.3936927   0.93589246  2.0046651  -0.19        0.38      ]]
a(144)=[ 0.98164153 -0.18869737]
u(144)=[0.9067416 0.5930115]
z(145)=[[ 1.3617398  1.0005519  2.0598862 -0.2        0.37     ]]
a(145)=[ 0.9861054 -0.3877293]
u(145)=[0.93996865 0.50895107]
z(146)=[[ 1.3349085  1.1030873  2.1036906 -0.21       0.36     ]]
a(146)=[ 0.98868185 -0.49556115]
u(146)=[0.9501347  0.34745386]
z(147)=[[ 1.2714875  1.1927656  2.153243  -0.22       0.35     ]]
a(147)=[ 0.98137665 -0.74361444]
u(147)=[0.9613308  0.16245821]
z(148)=[[ 1.2184042  1.261119   2.176756  -0.23       0.34     ]]
a(148)=[ 0.93789655 -0.86552083]
u(148)=[0.9665444  0.11868265]
z(149)=[[ 1.1633698  1.332591   2.1948557 -0.24       0.33     ]]
a(149)=[ 0.7986131 -

z(210)=[[ 1.5509818  1.0252324  1.6443673  0.47      -0.28     ]]
a(210)=[-0.90047413  0.68762934]
u(210)=[-0.99881023 -0.11708469]
z(211)=[[ 1.5578349  0.9206069  1.630303   0.46      -0.29     ]]
a(211)=[-0.8414279  0.7047031]
u(211)=[-0.99758625  0.08310446]
z(212)=[[ 1.5557817  0.8269639  1.6331844  0.45      -0.3      ]]
a(212)=[-0.8037534   0.70996165]
u(212)=[-0.99278     0.30040216]
z(213)=[[ 1.5569855  0.7115342  1.6609644  0.44      -0.31     ]]
a(213)=[-0.8153337   0.62770575]
u(213)=[-0.98259574  0.4439966 ]
z(214)=[[ 1.5778033  0.5944767  1.7091378  0.43      -0.32     ]]
a(214)=[-0.8436973   0.47812015]
u(214)=[-0.9663858  0.5202783]
z(215)=[[ 1.588751    0.50861233  1.7740277   0.42       -0.33      ]]
a(215)=[-0.86248714  0.20122597]
u(215)=[-0.93754035  0.64527434]
z(216)=[[ 1.6019502   0.40695044  1.8409883   0.41       -0.34      ]]
a(216)=[-0.84985363 -0.13120614]
u(216)=[-0.91308564  0.6941988 ]
z(217)=[[ 1.6316059   0.31192964  1.9136223   0.4        -0.35      ]]

z(273)=[[ 2.3193648   0.92880183  1.9231415  -0.16        0.4       ]]
a(273)=[0.9823894 0.9511616]
u(273)=[0.9752224  0.86035836]
z(274)=[[ 2.2806652  1.0317489  2.023505  -0.17       0.39     ]]
a(274)=[0.96041054 0.93578726]
u(274)=[0.9853816 0.8352265]
z(275)=[[ 2.2460382  1.1189264  2.1075287 -0.18       0.38     ]]
a(275)=[0.93899196 0.8755281 ]
u(275)=[0.9908464 0.8009115]
z(276)=[[ 2.2095654  1.212516   2.1785896 -0.19       0.37     ]]
a(276)=[0.9032464 0.7386385]
u(276)=[0.99408877 0.792053  ]
z(277)=[[2.139481  1.31167   2.2405329 0.5       0.36     ]]
a(277)=[-0.6615372  0.9788894]
u(277)=[0.9950384  0.86240137]
z(278)=[[2.0623481 1.3936874 2.3274658 0.5       0.35     ]]
a(278)=[-0.4678719  0.9413475]
u(278)=[0.9929623  0.91630876]
z(279)=[[1.9871508 1.4554734 2.4172337 0.5       0.34     ]]
a(279)=[-0.31445274  0.82173693]
u(279)=[0.9905282 0.9415837]
z(280)=[[1.9047709 1.5345496 2.5154386 0.5       0.33     ]]
a(280)=[-0.2576103  0.204269 ]
u(280)=[0.9823894 0.9511616]
z

...

z(686)=[[ 1.9630271  -0.41388288  2.5839148   0.1         0.5       ]]
a(686)=[ 0.2165428  -0.93005073]
u(686)=[-0.83514154 -0.95115954]
z(687)=[[ 2.0293162  -0.46648636  2.489556    0.09        0.5       ]]
a(687)=[ 0.73800486 -0.8584846 ]
u(687)=[-0.8521508  -0.95881313]
z(688)=[[ 2.099292  -0.5053148  2.4166253  0.08       0.5      ]]
a(688)=[ 0.86408526 -0.74281454]
u(688)=[-0.8524881 -0.9609972]
z(689)=[[ 2.1662512 -0.5670505  2.3203619  0.07       0.5      ]]
a(689)=[ 0.94256115 -0.67549384]
u(689)=[-0.8307846 -0.9596646]
z(690)=[[ 2.219191  -0.6320862  2.237501   0.06       0.5      ]]
a(690)=[ 0.96812195 -0.6290525 ]
u(690)=[-0.79547656 -0.95661086]
z(691)=[[ 2.2780354 -0.6932776  2.1706316  0.05       0.5      ]]
a(691)=[ 0.97816586 -0.541415  ]
u(691)=[-0.6739578 -0.9514983]
z(692)=[[ 2.3198204  -0.75597334  2.0882943   0.04        0.5       ]]
a(692)=[ 0.98463   -0.5475917]
u(692)=[-0.38277733 -0.95602787]
z(693)=[[ 2.348793   -0.79305315  1.9895389   0.03        0.5       ]

z(753)=[[ 1.7486644  2.041678   2.2444668  0.5       -0.02     ]]
a(753)=[-0.95473796  0.35257813]
u(753)=[0.5973661 0.1491067]
z(754)=[[ 1.7197381  2.0769217  2.255992   0.5       -0.03     ]]
a(754)=[-0.96476257  0.31751236]
u(754)=[0.13171962 0.10856728]
z(755)=[[ 1.7011348  2.0919533  2.2649152  0.5       -0.04     ]]
a(755)=[-0.97084534  0.3049255 ]
u(755)=[-0.33714592  0.14472695]
z(756)=[[ 1.7270191  2.0643926  2.26272    0.5       -0.05     ]]
a(756)=[-0.97198474  0.33873156]
u(756)=[-0.6770326   0.25501886]
z(757)=[[ 1.7627478  2.0208972  2.288878   0.5       -0.06     ]]
a(757)=[-0.97135746  0.37816265]
u(757)=[-0.8251688   0.39631686]
z(758)=[[ 1.8121023  1.9654933  2.3198655  0.5       -0.07     ]]
a(758)=[-0.96369666  0.42015263]
u(758)=[-0.90271497  0.3457251 ]
z(759)=[[ 1.8894403  1.90167    2.3548682  0.5       -0.08     ]]
a(759)=[-0.9109138   0.36278614]
u(759)=[-0.93611705  0.3637927 ]
z(760)=[[ 1.9433454  1.8363585  2.3815184  0.5       -0.09     ]]
a(760)=[-0.76498

z(816)=[[ 2.5057886  -0.28710565  1.7429019   0.23        0.46      ]]
a(816)=[0.81843007 0.9470871 ]
u(816)=[0.7590619 0.9803191]
z(817)=[[ 2.5016048  -0.21384268  1.8475424   0.22        0.45      ]]
a(817)=[0.84173787 0.9160643 ]
u(817)=[0.7435671 0.9782457]
z(818)=[[ 2.4834385  -0.14054842  1.9297204   0.21        0.5       ]]
a(818)=[0.8806466 0.9407273]
u(818)=[0.74905837 0.9719284 ]
z(819)=[[ 2.4664624  -0.07365266  2.0366318   0.2         0.5       ]]
a(819)=[0.89594597 0.92461133]
u(819)=[0.7762828 0.9734626]
z(820)=[[ 2.420967   -0.00535679  2.1573546   0.19        0.5       ]]
a(820)=[0.8991655 0.8943468]
u(820)=[0.8103592  0.97060156]
z(821)=[[2.3673813  0.07122053 2.2613423  0.18       0.49      ]]
a(821)=[0.80053353 0.7036489 ]
u(821)=[0.8051424 0.9652507]
z(822)=[[2.3021085  0.13327509 2.3725386  0.17       0.48      ]]
a(822)=[0.7484256  0.16673334]
u(822)=[0.8207718 0.9558093]
z(823)=[[2.2403302  0.20414269 2.490384   0.16       0.47      ]]
a(823)=[ 0.6360233  -0.5130

z(881)=[[ 2.5877478  1.2426419  1.6369058  0.45      -0.11     ]]
a(881)=[-0.3647197   0.04384193]
u(881)=[-0.9248828 -0.674753 ]
z(882)=[[ 2.5967247  1.1440711  1.586832   0.44      -0.12     ]]
a(882)=[-0.16376399  0.18377325]
u(882)=[-0.8784406  -0.70558155]
z(883)=[[ 2.6056402  1.0513018  1.514049   0.43      -0.13     ]]
a(883)=[-0.04723166  0.3084148 ]
u(883)=[-0.82486063 -0.6876732 ]
z(884)=[[ 2.5999477  0.9752575  1.4317641  0.42      -0.14     ]]
a(884)=[-0.11155427  0.45148543]
u(884)=[-0.7838607 -0.68886  ]
z(885)=[[ 2.5846639  0.8809744  1.3672872  0.41      -0.15     ]]
a(885)=[-0.2373059   0.46768478]
u(885)=[-0.7062173 -0.6256076]
z(886)=[[ 2.5632453   0.81014186  1.2981062   0.4        -0.16      ]]
a(886)=[-0.42163354  0.52795786]
u(886)=[-0.6357341 -0.4268996]
z(887)=[[ 2.5416958   0.74999684  1.25575     0.39       -0.17      ]]
a(887)=[-0.41081104  0.5820875 ]
u(887)=[-0.52686614 -0.16631098]
z(888)=[[ 2.5119932  0.6953394  1.2425458  0.38      -0.18     ]]
a(888)=[

z(947)=[[1.830543  1.781737  2.3355365 0.5       0.21     ]]
a(947)=[0.07030971 0.05239243]
u(947)=[-0.5924643   0.87773144]
z(948)=[[1.8672335 1.7399521 2.442266  0.5       0.2      ]]
a(948)=[ 0.15817235 -0.32111156]
u(948)=[-0.35604173  0.8097706 ]
z(949)=[[1.8911307 1.6985393 2.5311172 0.5       0.19     ]]
a(949)=[ 0.20933528 -0.5459057 ]
u(949)=[-0.03763877  0.7001661 ]
z(950)=[[1.8920027 1.6945332 2.5963607 0.5       0.18     ]]
a(950)=[ 0.1746135 -0.6380435]
u(950)=[0.1496308 0.5830145]
z(951)=[[1.895189  1.6829935 2.6613405 0.5       0.17     ]]
a(951)=[-0.01988779 -0.66957295]
u(951)=[0.00656019 0.3301978 ]
z(952)=[[1.9047761 1.685445  2.6941555 0.5       0.16     ]]
a(952)=[-0.04515417 -0.70637286]
u(952)=[-0.03241787  0.25919986]
z(953)=[[1.9158827 1.6810708 2.7040036 0.5       0.15     ]]
a(953)=[ 0.00259736 -0.7290518 ]
u(953)=[-0.06041135  0.2571481 ]
z(954)=[[1.9261533 1.6696459 2.7327957 0.5       0.14     ]]
a(954)=[ 0.01867156 -0.7578838 ]
u(954)=[0.07030971 0.052392

In [8]:
stl_return_val

-55.74702452907696